**updates on this version:**

--changed patient arrival to allow for different input each day of the week and for each hour of each day (input is patients/hr at this time, no st. dev or quartiles yet) (previous version had only diff. input for diff. days of the week, but not broken down by hour)

**next:** 


--Fix boxplot order

--Inventory buildup diagram of beds

--Add data

--Refine output (summary table)


In [ ]:
# The !pip command is used to install packages in Python using the pip package manager.
!pip install simpy # simpy is a discrete-event simulation library used in the simulation code
!pip install matplotlib # matplotlib is a popular plotting library for Python 
!pip install prettytable #prettytable is a library for generating ASCII tables in Python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import simpy # simpy: a Python library for discrete-event simulation
import random #random: a built-in Python library for generating random numbers
import matplotlib.pyplot as plt # matplotlib.pyplot: a Python library for creating plots and visualizations
import numpy as np #numpy: a Python library for numerical computing
from prettytable import PrettyTable #prettytable: a Python library for creating table
import datetime #datetime: a built-in Python library for working with dates and times
import pandas as pd #pandas: a Python library for data manipulation and analysis
from contextlib import ExitStack # imports the ExitStack context manager from the Python standard library's contextlib module


In [ ]:
# Seed for random number generator (for the Python's built-in pseudo-random number generator, which is used by the random module)
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

# Simulation parameters to define the setting of the simulation
SIM_TIME = 10080  # Simulation time in minutes (1 week, because 1 week=60x24x7=10080 minutes)
CUBICLE_BEDS = 75 # There are 50 cubicle beds + 25 hallway beds
WARD_BEDS = 37 # There are 25 beds in the ward + 12 beds in the overflow rooms that belong to wards
WEEKDAY_DOCTORS = 1000
WEEKEND_DOCTORS = 1000
WEEKDAY_NURSES = 1000
WEEKEND_NURSES = 1000
WEEKDAY_TRIAGENURSES = 1000
WEEKEND_TRIAGENURSES = 1000

NURSE_AVG_TIME = 10  # average time a nurse spends with a patient in minutes
TRIAGENURSES_AVG_TIME = 10 
DOCTOR_AVG_TIME = 20  # average time a doctor spends with a patient in minutes

SIMULATION_STARTTIMESTAMP_STRING = "2023-01-20 15:00:00"
TIME_FMT = "%Y-%m-%d %H:%M:%S"


# Define resource requirements for each triage level
# dictionary that holds the required resources for each of the 5 triage levels (used later in the simulation to allocate resources to patients based on their triage level)
TRIAGE_RESOURCES = {
    1: {'doctors': 2, 'nurses': 2, 'ed cubicle beds': 1, 'tests': 2},
    2: {'doctors': 1, 'nurses': 2, 'ed cubicle beds': 1, 'tests': 1},
    3: {'doctors': 1, 'nurses': 1, 'ed cubicle beds': 1, 'tests': 1},
    4: {'doctors': 1, 'nurses': 1, 'ed cubicle beds': 1, 'tests': 1},
    5: {'doctors': 1, 'nurses': 1, 'ed cubicle beds': 1, 'tests': 0},
}

# Define admission probabilities for each triage level
ADMISSION_PROBABILITIES = {
    1: 0.7643,  # probability of admission for severe cases
    2: 0.4784,  # probability of admission for moderate cases
    3: 0.2133,  # probability of admission for mild cases
    4: 0.0195,  # probability of admission for very mild cases
    5: 0.0097,  # probability of admission for least severe cases
}

# Get the admission probability based on triage level
def get_admission_probability(triage_level):
    return ADMISSION_PROBABILITIES[triage_level]


# Define triage treatment times for each triage level
def get_triage_times(averages, standard_deviations):
    triage_times = {}
    for level in range(1, 6):
        average = averages[level]
        standard_deviation = standard_deviations[level]
        min_time = round(random.gauss(average, standard_deviation))
        max_time = round(random.gauss(average, standard_deviation))
        triage_times[level] = (min(min_time, max_time), max(min_time, max_time))
    return triage_times

averages = {
    1: 10,   # average treatment time for severe cases
    2: 15,   # average treatment time for moderate cases
    3: 20,   # average treatment time for mild cases
    4: 20,   # average treatment time for very mild cases
    5: 22,   # average treatment time for least severe cases
}

standard_deviations = {
    1: 2,   # standard deviation of treatment times for severe cases
    2: 3,   # standard deviation of treatment times for moderate cases
    3: 4,   # standard deviation of treatment times for mild cases
    4: 4,   # standard deviation of treatment times for very mild cases
    5: 2,   # standard deviation of treatment times for least severe cases
}

TRIAGE_TIMES = get_triage_times(averages, standard_deviations)


# Define nurse treatment times for each triage level
def get_nurse_times(averages, standard_deviations):
    nurse_times = {}
    for level in range(1, 6):
        average = averages[level]
        standard_deviation = standard_deviations[level]
        min_time = round(random.gauss(average, standard_deviation))
        max_time = round(random.gauss(average, standard_deviation))
        nurse_times[level] = (min(min_time, max_time), max(min_time, max_time))
    return nurse_times

averages = {
    1: 10,   # average treatment time for severe cases
    2: 10,   # average treatment time for moderate cases
    3: 10,   # average treatment time for mild cases
    4: 10,   # average treatment time for very mild cases
    5: 10,   # average treatment time for least severe cases
}

standard_deviations = {
    1: 5,   # standard deviation of treatment times for severe cases
    2: 5,   # standard deviation of treatment times for moderate cases
    3: 5,   # standard deviation of treatment times for mild cases
    4: 5,   # standard deviation of treatment times for very mild cases
    5: 5,   # standard deviation of treatment times for least severe cases
}

NURSE_TIMES = get_nurse_times(averages, standard_deviations)


# Define doctor treatment times for each triage level
def get_doctor_times(averages, standard_deviations):
    doctor_times = {}
    for level in range(1, 6):
        average = averages[level]
        standard_deviation = standard_deviations[level]
        min_time = round(random.gauss(average, standard_deviation))
        max_time = round(random.gauss(average, standard_deviation))
        doctor_times[level] = (min(min_time, max_time), max(min_time, max_time))
    return doctor_times

averages = {
    1: 15,   # average treatment time for severe cases
    2: 15,   # average treatment time for moderate cases
    3: 15,   # average treatment time for mild cases
    4: 15,   # average treatment time for very mild cases
    5: 15,   # average treatment time for least severe cases
}

standard_deviations = {
    1: 5,   # standard deviation of treatment times for severe cases
    2: 5,   # standard deviation of treatment times for moderate cases
    3: 5,   # standard deviation of treatment times for mild cases
    4: 5,   # standard deviation of treatment times for very mild cases
    5: 5,   # standard deviation of treatment times for least severe cases
}

DOCTOR_TIMES = get_doctor_times(averages, standard_deviations)


# Define medical tests times for each triage level
def get_test_times(averages, standard_deviations):
    test_times = {}
    for level in range(1, 6):
        average = averages[level]
        standard_deviation = standard_deviations[level]
        min_time = round(random.gauss(average, standard_deviation))
        max_time = round(random.gauss(average, standard_deviation))
        test_times[level] = (min(min_time, max_time), max(min_time, max_time))
    return test_times

averages = {
    1: 25,   # average test time for severe cases
    2: 20,   # average test time for moderate cases
    3: 15,   # average test time for mild cases
    4: 10,   # average test time for very mild cases
    5: 0,    # least severe cases do not require any tests
}

standard_deviations = {
    1: 5,   # standard deviation of test times for severe cases
    2: 4,   # standard deviation of test times for moderate cases
    3: 3,   # standard deviation of test times for mild cases
    4: 2,   # standard deviation of test times for very mild cases
    5: 0,   # least severe cases do not require any tests
}

TEST_TIMES = get_test_times(averages, standard_deviations)



# Define nurse ward treatment times for each triage level
def get_nurseward_times(averages, standard_deviations):
    nurseward_times = {}
    for level in range(1, 6):
        average = averages[level]
        standard_deviation = standard_deviations[level]
        min_time = round(random.gauss(average, standard_deviation))
        max_time = round(random.gauss(average, standard_deviation))
        nurseward_times[level] = (min(min_time, max_time), max(min_time, max_time))
    return nurseward_times

averages = {
    1: 15,   # average treatment time for severe cases in nurse ward
    2: 15,   # average treatment time for moderate cases in nurse ward
    3: 15,   # average treatment time for mild cases in nurse ward
    4: 15,   # average treatment time for very mild cases in nurse ward
    5: 15,   # average treatment time for least severe cases in nurse ward
}

standard_deviations = {
    1: 5,   # standard deviation of treatment times for severe cases in nurse ward
    2: 5,   # standard deviation of treatment times for moderate cases in nurse ward
    3: 5,   # standard deviation of treatment times for mild cases in nurse ward
    4: 5,   # standard deviation of treatment times for very mild cases in nurse ward
    5: 5,   # standard deviation of treatment times for least severe cases in nurse ward
}

NURSEWARD_TIMES = get_nurseward_times(averages, standard_deviations)


# Define doctor ward treatment times for each triage level
def get_doctorward_times(averages, standard_deviations):
    doctorward_times = {}
    for level in range(1, 6):
        average = averages[level]
        standard_deviation = standard_deviations[level]
        min_time = round(random.gauss(average, standard_deviation))
        max_time = round(random.gauss(average, standard_deviation))
        doctorward_times[level] = (min(min_time, max_time), max(min_time, max_time))
    return doctorward_times

averages = {
    1: 15,   # average treatment time for severe cases in doctor ward
    2: 15,   # average treatment time for moderate cases in doctor ward
    3: 15,   # average treatment time for mild cases in doctor ward
    4: 15,   # average treatment time for very mild cases in doctor ward
    5: 15,   # average treatment time for least severe cases in doctor ward
}

standard_deviations = {
    1: 5,   # standard deviation of treatment times for severe cases in doctor ward
    2: 5,   # standard deviation of treatment times for moderate cases in doctor ward
    3: 5,   # standard deviation of treatment times for mild cases in doctor ward
    4: 5,   # standard deviation of treatment times for very mild cases in doctor ward
    5: 5,   # standard deviation of treatment times for least severe cases in doctor ward
}

DOCTORWARD_TIMES = get_doctorward_times(averages, standard_deviations)


In [ ]:
def get_env_datetime(simulation_time_minutes_since_start, simulation_start_timestamp_string=SIMULATION_STARTTIMESTAMP_STRING, time_fmt = TIME_FMT):
  """get environment time as a datetime"""
  # convert start timestamp from string to datetime
  simulation_start_datetime = datetime.datetime.strptime(simulation_start_timestamp_string, time_fmt)
  simulation_datetime = simulation_start_datetime + datetime.timedelta(minutes=simulation_time_minutes_since_start)
  return simulation_datetime # this is a datetime.datetime object

def get_env_timestamp_string(simulation_time_minutes_since_start, simulation_start_timestamp_string=SIMULATION_STARTTIMESTAMP_STRING, time_fmt = TIME_FMT):
  """get environment time as a string"""
  simulation_datetime = get_env_datetime(simulation_time_minutes_since_start, simulation_start_timestamp_string, time_fmt)
  simulation_timestamp_string = simulation_datetime.strftime(time_fmt)
  return simulation_timestamp_string


# e.g. get_env_timestamp_string(env.now) will return a string like '2023-01-20 15:05:00'

In [ ]:
# Get the capacity of doctors based on the current time
# takes the current time env.now (in minutes) in the simulation environment as input and returns the capacity of doctors based on the current day and time of the week
def get_doctors_capacity(env): # get_doctors_capacity(env) returns the capacity of doctors based on the current time
    current_time = env.now % 10080 #minutes/week
    day = current_time // 1440
    if day < 5:
        return WEEKDAY_DOCTORS
    else:
        return WEEKEND_DOCTORS


# Get the capacity of nurses based on the current time
# takes the current time env.now (in minutes) in the simulation environment as input and returns the capacity of nurses based on the current day and time of the week
def get_nurses_capacity(env): # get_nurse_capacity(env) returns the capacity of doctors based on the current time
    current_time = env.now % 10080 #minutes/week
    day = current_time // 1440
    if day < 5:
        return WEEKDAY_NURSES
    else:
        return WEEKEND_NURSES

# Get the capacity of triagenurses based on the current time
# takes the current time env.now (in minutes) in the simulation environment as input and returns the capacity of nurses based on the current day and time of the week
def get_triagenurses_capacity(env): # get_nurse_capacity(env) returns the capacity of doctors based on the current time
    current_time = env.now % 10080 #minutes/week
    day = current_time // 1440
    if day < 5:
        return WEEKDAY_TRIAGENURSES
    else:
        return WEEKEND_TRIAGENURSES

def get_bed_capacity(env):
    current_time = env.now % 10080  # minutes/week
    day = current_time // 1440  # minutes/day

    if day < 5:  # Weekday
        return {
            'cubicle beds': 50,
            'ward beds': 50
        }
    else:  # Weekend
        return {
            'cubicle beds': 30,
            'ward beds': 30
        }

        
# Define a function to print timestamped events
def print_event(event):
    print(f'{env.now:,.0f} minutes: {event}')


In [ ]:
# Initialize simulation environment
# creates a new simulation environment using the simpy module
# The simulation environment is a container that holds all the processes, resources, and events that make up the simulation. It serves as the backbone of the simulation and provides a framework for simulating the behavior of the system over time
env = simpy.Environment(initial_time=0)

# Initialize resources
ed_cubiclebed = simpy.PriorityResource(env, capacity=CUBICLE_BEDS)  
ward_beds = simpy.PriorityResource(env, capacity=WARD_BEDS)
doctors = simpy.PriorityResource(env, capacity=get_doctors_capacity(env))
nurses = simpy.PriorityResource (env, capacity=get_nurses_capacity (env))
triagenurses = simpy.PriorityResource (env, capacity=get_triagenurses_capacity (env))


# Define a queue for each triage level
triage_queues = {1: simpy.PriorityResource(env), 2: simpy.PriorityResource(env), 3: simpy.PriorityResource(env), 4: simpy.PriorityResource(env), 5: simpy.PriorityResource(env)}




In [ ]:
import random

class Patient:
    def __init__(self, number):
        self.number = number
        self.triage_level = None
        self.discharge_time = None
        self.arrival_time = None
        self.triage_level = None
        self.triage_nurse_time = None
        self.cubiclebed_time = None
        self.nurse_time = None
        self.doctor_time = None
        self.tests_time = None
        self.ward_time = None
        self.nurseward_time = None
        self.doctorward_time = None 

    def __repr__(self):
        return f"Patient #{self.number}"

def get_patients_arrival_rate(env):
    current_time = env.now % 10080  # minutes/week
    day = int(current_time // 1440)  # minutes/day (converted to integer)
    hour = int((current_time % 1440) // 60)  # minutes/hour (converted to integer)

    # Define arrival rates for each day of the week and hour of the day (in patients per hour)
    arrival_rates = [
        [5, 8, 12, 10, 15, 18, 20, 22, 25, 23, 20, 15, 10, 8, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],  # Day 1
        [6, 8, 10, 14, 16, 18, 20, 22, 24, 23, 21, 18, 16, 14, 12, 10, 8, 6, 5, 5, 5, 5, 5, 5],  # Day 2
        [5, 6, 8, 10, 12, 14, 16, 18, 20, 21, 22, 22, 20, 18, 16, 14, 12, 10, 8, 6, 5, 5, 5, 5],  # Day 3
        [5, 6, 8, 10, 12, 14, 16, 18, 20, 21, 22, 22, 20, 18, 16, 14, 12, 10, 8, 6, 5, 5, 5, 5],  # Day 4
        [5, 6, 8, 10, 12, 14, 16, 18, 20, 21, 22, 22, 20, 18, 16, 14, 12, 10, 8, 6, 5, 5, 5, 5],  # Day 5
        [5, 6, 8, 10, 12, 14, 16, 18, 20, 21, 22, 22, 20, 18, 16, 14, 12, 10, 8, 6, 5, 5, 5, 5],  # Day 6
        [5, 6, 8, 10, 12, 14, 16, 18, 20, 21, 22, 22, 20, 18, 16, 14, 12, 10, 8, 6, 5, 5, 5, 5],  # Day 7
  
    ]

    # Get the arrival rate based on the current day and hour
    arrival_rate = arrival_rates[day][hour]

    return arrival_rate

# Define patient arrival process
def patient_arrival(env, patient_data):
    patient_num = 0

    while True:
        # Calculate the arrival rate based on the current time
        arrival_rate = get_patients_arrival_rate(env)

        # Schedule next patient arrival based on the arrival rate
        interarrival_time = random.expovariate(arrival_rate)
        yield env.timeout(interarrival_time)

        # Increment patient ID and start triage process
        patient_num += 1

        patient = Patient(patient_num)
        patient.arrival_time = env.now
        patient.arrival_time_str = get_env_timestamp_string(env.now)
        print(f"{patient} arrives at {patient.arrival_time_str}")

        patient_data.append(patient)

        env.process(triage(env, patient))



In [ ]:
#triage process
def triage(env, patient):
    triage_levels = [1, 2, 3, 4, 5] # ESI levels
    triage_weights = [0.0026, 0.2765, 0.6103, 0.149, 0.0057]  #probability of patients in each triage level
    triage_level = random.choices(triage_levels, weights=triage_weights)[0] 
    patient.triage_level = triage_level

    with triagenurses.request(priority=triage_level) as req:
        yield req 

        patient.triage_nurse_time = env.now
        triagenurses_time_str = get_env_timestamp_string(env.now)
        print(f"{patient} assigned triage nurse at {triagenurses_time_str}")

        triage_time = random.randint(*TRIAGE_TIMES[triage_level])
        yield env.timeout(triage_time) # make the environment consume some amount of time

        patient.triage_time = env.now
        triage_time_str = get_env_timestamp_string(env.now)
        print(f"{patient} completed triage at {triage_time_str}")

        # Call the next stage of the simulation for the patient with the triage level and env parameters
        env.process(emergency_department(env, patient, triage_level))

    

In [ ]:
def func():
  with beds.request(priority=triage_level) as req:
    yield req

In [ ]:
func()

<generator object func at 0x7faa08fb1cb0>

In [ ]:
### NEED TO DEFINE AN ED TIME 
ed_cubicle_time = 10


In [ ]:
# Emergency department
def emergency_department(env, patient, triage_level):
    # Determine the number of resources required based on triage level
    # The TRIAGE_RESOURCES dictionary contains a mapping between the different triage levels (1-5) and the resources required for each level
    num_doctors = TRIAGE_RESOURCES[triage_level]['doctors']
    num_nurses = TRIAGE_RESOURCES[triage_level]['nurses']
    num_edcubicle_beds = TRIAGE_RESOURCES[triage_level]['ed cubicle beds']
    num_tests = TRIAGE_RESOURCES[triage_level]['tests']

    # Send patient to queue for ED cubicle
    num_ed_cubiclebed_available = ed_cubiclebed.capacity - ed_cubiclebed.count
    
    print('NUM_ED_CUBICLEBEDS_Available'+str(num_ed_cubiclebed_available))
    

    with ed_cubiclebed.request(priority=triage_level) as req:
      yield req
      num_ed_cubiclebed_available = ed_cubiclebed.capacity - ed_cubiclebed.count
      cubicle_time = env.now
      patient.cubicle_time = env.now
      cubicle_time_str = get_env_timestamp_string(cubicle_time)
      print(f"{patient} enters ED cubicle at {cubicle_time_str}")
      env.timeout(ed_cubicle_time)

    with ExitStack() as stack0:
        requests = [stack0.enter_context(ed_cubiclebed.request(priority=triage_level)) for _ in range(1)]
        for req0 in requests:
          yield req0
        cubicle_time = env.now
        patient.cubicle_time = env.now
        cubicle_time_str = get_env_timestamp_string(cubicle_time)
        print(f"{patient} enters ED cubicle at {cubicle_time_str}")

    # Wait for nurse(s) to check patient
    with ExitStack() as stack1:
        requests = [stack1.enter_context(nurses.request(priority=triage_level)) for _ in range(num_nurses)]
        for req1 in requests:
            yield req1
        nurse_time = env.now
        patient.nurse_time = nurse_time
        nurse_time_str = get_env_timestamp_string(nurse_time)
        print(f"Nurse checks {patient} at {nurse_time_str}")   
        print(f"NURSES AVAILABLE: {nurses.capacity - nurses.count}") 

        nurse_time = random.randint(*NURSE_TIMES[triage_level])
        yield env.timeout(nurse_time) # make the environment consume some amount of time

        # waits for a specified number of doctors to assess the patient. The number of doctors required is determined by the patient's triage level
        # This code creates a context for the doctors resource pool and requests the specified number of doctors to assess the patient
        with ExitStack() as stack2:
            requests = [stack2.enter_context(doctors.request(priority=triage_level)) for _ in range(num_doctors)] 
            for req2 in requests: # The for loop iterates through the requests list and yields each req2 object (the code waits for each doctor to become available before continuing to the next line of code. Once all the doctors become available, the patient is assessed by the doctors)
                yield req2
            doctor_time = env.now # timestamp when the doctors assess the patient
            patient.doctor_time = doctor_time
            doctor_time_str = get_env_timestamp_string(doctor_time) # format time
            print(f"Doctors assess {patient} at {doctor_time_str}")
            print(f"DOCTORS AVAILABLE: {doctors.capacity - doctors.count}")

            doctor_time = random.randint(*DOCTOR_TIMES[triage_level])
            yield env.timeout(doctor_time) # make the environment consume some amount of time

            # Wait for nurse(s) to perform tests
            # creates a context for the nurses resource pool and requests the specified number of nurses to perform tests on the patient
            with ExitStack() as stack3:
                requests = [stack3.enter_context(nurses.request(priority=triage_level)) for _ in range(num_nurses)]
                for req3 in requests: # The for loop iterates through the requests list and yields each req3 object (the code waits for each nurse to become available before continuing to the next line of code. Once all the nurses become available, the patient is tested by the nurses)
                    yield req3
                # Perform medical test (only on weekdays)
                if num_tests > 0 and get_env_datetime(env.now).weekday() < 5: #only perform tests on weekdays
                    test_time = random.randint(*TEST_TIMES[triage_level])
                    yield env.timeout(test_time)
                
                tests_time = env.now # timestamp when the tests are performed
                patient.tests_time = tests_time
                tests_time_str = get_env_timestamp_string(tests_time) # format time
                print(f"Nurses perform tests on {patient} at {tests_time_str}")
                
                # Treat patient
                # waits for the appropriate number of nurses to become available to treat the patient. It uses the ExitStack() context manager to manage the requests for the nurses, which are entered in the list requests
                # number of nurses required to treat the patient depends on the severity level assigned during triage (triage_level). This is stored in the variable num_nurses, which was assigned the value of the corresponding entry in the TRIAGE_RESOURCES dictionary
                nurse_time = random.expovariate(1 / NURSE_AVG_TIME)
                doctor_time = random.expovariate(1 / DOCTOR_AVG_TIME)
                treat_time = env.now # timestamp patient is treated
                treat_time_str = get_env_timestamp_string(treat_time) # format time
                print(f"{num_nurses} nurse(s) and {num_doctors} doctor(s) treat patient {patient} for {nurse_time:.2f} min (nurse) + {doctor_time:.2f} min (doctor) at {treat_time_str}")

                # yield env.timeout(nurse_time + doctor_time) is a way to make the patient wait for a certain amount of time in the simulation. The env.timeout() method is provided by the simpy package and creates a timeout event that will trigger after a certain amount of time has elapsed
                # nurse_time and doctor_time represent the time that the patient spends being checked and treated by nurses and doctors
                # The yield statement is used to suspend the execution of the patient process until the timeout event occurs
                yield env.timeout(nurse_time + doctor_time)

                # Call the next stage of the simulation for the patient with the triage level and env parameters
                env.process(hospital_ward(env, patient, triage_level, num_doctors, num_nurses, num_tests))

In [ ]:
def hospital_ward(env, patient, triage_level, num_doctors, num_nurses, num_tests):
    # Decide whether to admit or discharge patient
    # Check if the patient is admitted based on the admission probability
    if random.random() < get_admission_probability(triage_level):
        # Send patient to queue for ward bed
            ward_time = env.now
            patient.ward_time = ward_time
            ward_time_str = get_env_timestamp_string(ward_time)
            print(f"{patient} admitted to ward at {ward_time_str}")

            # Request nurses and doctors to treat patient in ward
            with ExitStack() as stack5:
                requests = [stack5.enter_context(nurses.request(priority=triage_level)) for _ in range(num_nurses)]
                for req5 in requests:
                    yield req5

                nurseward_time = env.now
                patient.nurseward_time = nurseward_time
                nurseward_time_str = get_env_timestamp_string(nurseward_time)
                print(f"{patient} nurse(s) treat patient {patient} in ward at {nurseward_time_str}")

                nurseward_time = random.randint(*NURSEWARD_TIMES[triage_level])
                yield env.timeout(nurseward_time) # make the environment consume some amount of time

                with ExitStack() as stack6:
                    requests = [stack6.enter_context(doctors.request(priority=triage_level)) for _ in range(num_doctors)]
                    for req6 in requests:
                        yield req6

                    doctorward_time = env.now
                    patient.doctorward_time = doctorward_time
                    doctorward_time_str = get_env_timestamp_string(doctorward_time)
                    print(f"{num_doctors} doctor(s) treat patient {patient} in ward at {doctorward_time_str}")

                    doctorward_time = random.randint(*DOCTORWARD_TIMES[triage_level])
                    yield env.timeout(doctorward_time) # make the environment consume some amount of time
            
            discharge_delay = random.uniform(10, 20)
            yield env.timeout(discharge_delay)

            discharge_time = env.now
            patient.discharge_time = discharge_time
            discharge_time_str = get_env_timestamp_string(discharge_time)
            print(f"Patient {patient} treated and discharged from ward at {discharge_time_str}")

            physical_exit_delay = random.uniform(10, 20)
            yield env.timeout(physical_exit_delay)

            physically_exit_time = env.now
            patient.physically_exit_time = physically_exit_time
            physically_exit_time_str = get_env_timestamp_string(physically_exit_time)
            print(f"Patient {patient} physically leaves ward and at {physically_exit_time_str}")

            bed_turnover_delay = random.uniform(10, 20)
            yield env.timeout(bed_turnover_delay)

            bed_turnover_time = env.now
            patient.bed_turnover_time = bed_turnover_time
            bed_turnover_time_str = get_env_timestamp_string(bed_turnover_time)
            print(f"Bed is turned over and ready for next patient at {bed_turnover_time}")

            patient.process_time = patient.discharge_time - patient.arrival_time
            print(f'Patient {patient} total process time: {patient.process_time} minutes')

            # Add patient data to summary table
            patient_discharge_data.append([patient, triage_level, 'Discharged from ward', discharge_time])
    else:
        # Discharge process for patient not admitted to ED 
        discharge_delay = random.uniform(5, 10)
        yield env.timeout(discharge_delay)

        discharge_time = env.now
        patient.discharge_time = discharge_time
        discharge_time_str = get_env_timestamp_string(discharge_time)
        print(f"Patient {patient} not admitted to ward and discharged at {discharge_time_str}")

        physical_exit_delay = random.uniform(10, 20)
        yield env.timeout(physical_exit_delay)

        physically_exit_time = env.now
        patient.physically_exit_time = physically_exit_time
        physically_exit_time_str = get_env_timestamp_string(physically_exit_time)
        print(f"Patient {patient} physically leaves ward and at {physically_exit_time_str}")

        patient.process_time = patient.discharge_time - patient.arrival_time
        print(f'Patient {patient} total process time: {patient.process_time} minutes')

        # Add patient data to summary table
        patient_discharge_data.append([patient, triage_level, 'Discharged from triage', discharge_time])


In [ ]:
# Initialize patient summary table and wait time list
patient_data = [] # patient_data is an empty list that will later be used to store data on each patient such as their ID, triage level, and discharge time
patient_discharge_data = []
wait_time = [] # wait_time is an empty list that will later be used to store the total wait time for each patient in the simulation
patient_table = PrettyTable(['Patient', 'Triage Level', 'Discharge Time']) # patient_table is a table object created using the PrettyTable library. It has four columns: "Patient", "Triage Level", "Outcome", and "Discharge Time". This table will be used to display a summary of the simulation results after it has run


In [ ]:
# Start patient arrival process
# starts the patient arrival process by calling the patient_arrival function and passing the env object (which represents the simulation environment) as an argument
# The patient_arrival process is a generator function that generates a sequence of events representing the arrival of patients to the emergency department over time. By starting this process, the simulation is initiated and will continue to run until all patients have been treated and discharged
env.process(patient_arrival(env, patient_data))

<Process(patient_arrival) object at 0x7faa08fdda50>

In [ ]:
# Run simulation
env.run(until=SIM_TIME) # tells the simulation to keep running until the simulation time reaches the SIM_TIME, which is the duration of the simulation that was specified at the beginning of the script. During this time, patients will be arriving, going through the triage process, and receiving treatment until they are either discharged or admitted to the ward

Streaming output truncated to the last 5000 lines.
Patient #107914 arrives at 2023-01-26 20:46:53
Patient #107914 assigned triage nurse at 2023-01-26 20:46:53
Patient #107677 completed triage at 2023-01-26 20:47:01
NUM_ED_CUBICLEBEDS_Available75
Patient #107677 enters ED cubicle at 2023-01-26 20:47:01
Patient #107677 enters ED cubicle at 2023-01-26 20:47:01
Patient #107709 completed triage at 2023-01-26 20:47:03
NUM_ED_CUBICLEBEDS_Available75
Patient #107709 enters ED cubicle at 2023-01-26 20:47:03
Patient #107709 enters ED cubicle at 2023-01-26 20:47:03
Patient #107678 completed triage at 2023-01-26 20:47:03
NUM_ED_CUBICLEBEDS_Available75
Patient #107678 enters ED cubicle at 2023-01-26 20:47:03
Patient #107678 enters ED cubicle at 2023-01-26 20:47:03
Patient #107915 arrives at 2023-01-26 20:47:05
Patient #107915 assigned triage nurse at 2023-01-26 20:47:05
Patient #107916 arrives at 2023-01-26 20:47:05
Patient #107916 assigned triage nurse at 2023-01-26 20:47:05
Patient #107680 comple

In [ ]:
len(patient_data)

In [ ]:
patient_data[90].__dict__

In [ ]:
patient_data[65]

In [ ]:
# used to access the internal dictionary of the nth ([n]) element of the patient_data list
# patient_data[n].__dict__ would return a dictionary of all the attributes associated with the 10th element in the patient_data list. This can be useful for inspecting the attributes of an object or for dynamically accessing and modifying its properties
# this code will only work if the nth element in the patient_data list is an object that has attributes stored in its internal dictionary. If the 10th element is not an object or does not have an internal dictionary, this code will result in an AttributeError
patient_data[548].__dict__ 

In [ ]:
patient_list = []
for p in patient_data:
  patient_list.append(p.__dict__)

In [ ]:
patient_df = pd.DataFrame(patient_list)
patient_df.head()

In [ ]:
patient_df[patient_df.discharge_time.isna()]

In [ ]:
np.isnan(patient_df['process_time'][2315])

In [ ]:
for p in patient_discharge_data:
  if(p[0]=='Patient #91'):
    print(p)

In [ ]:
res = 0
for p in patient_data:
  if type(p)==dict:
    print(p.__dict__['received_bed'])
    if(p.__dict__['received_bed'] == False):
      print(p)
      res += 1

In [ ]:
res

In [ ]:
res = []
for p in patient_data:
  if type(p)==list:
    res.append(p[2])
    if p[0]=='Patient #50':
      print(p)
      break

In [ ]:
from collections import Counter
Counter(res)

In [ ]:
patient_data[100]

In [ ]:
#Populate patient summary table
# creates a pretty table using the PrettyTable library and populates it with the data collected during the simulation from the patient_data list
for patient in patient_data:
    patient_table.add_row((patient.number, patient.triage_level, patient.discharge_time)) # The add_row() method of the PrettyTable object adds a row of data to the table with the specified column headings. In this case, the column headings are Patient, Triage Level, and Discharge Time, and the data for each row is obtained from the patient_data list

In [ ]:
# Print the table
print(patient_table)# Print patient summary table


In [ ]:
np.isnan(patient_df['process_time'])

In [ ]:
for idx,row in patient_df.iterrows():
  print(row['process_time'])
  print(np.isnan(row['process_time']))

In [ ]:
# Generate chart of patient wait times by triage level
# generates a box plot showing the distribution of wait times for patients in each triage level
# It first creates an empty list for each of the 5 triage levels
# Then it iterates over the patient data list, which contains information about each patient's triage level and their discharge time
import matplotlib.pyplot as plt
from collections import defaultdict

triage_data = {}
for idx,row in patient_df.iterrows():
    if row['triage_level'] in triage_data and not np.isnan(row['process_time']):
      triage_data[row['triage_level']].append(row['process_time']) # For each patient, it appends their discharge time to the appropriate triage level list
    if row['triage_level'] not in triage_data and not np.isnan(row['process_time']):
      triage_data[row['triage_level']] = [row['process_time']]

plt.boxplot(triage_data)#, labels=["Triage " + str(i+1) for i in range(6)]) # plots a box plot using the triage_data lists
plt.title("Patient Wait Times by Triage Level")
plt.xlabel("Triage Level")
plt.ylabel("Wait Time (minutes)")
plt.show()

In [ ]:
from math import isnan
filtered_triage_data = {k:[elem for elem in v if elem is not np.nan] for k,v in triage_data.items()}

In [ ]:
triage_data

In [ ]:

Counter(filtered_triage_data[1])

In [ ]:
plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True
fig, ax = plt.subplots()
ax.boxplot(triage_data.values(), positions=[1,2,3,4,5])
ax.set_xticklabels(triage_data.keys())
plt.show()

In [ ]:
triage_data.keys()

In [ ]:
import numpy as np
np.nanmean(triage_data[1])

In [ ]:
triage_data[1][1:10]

In [ ]:
np.nanmean(triage_data[2])

In [ ]:
np.nanmean(triage_data[3])

In [ ]:
np.nanmean(triage_data[4])

In [ ]:
np.nanmean(triage_data[5])